In [2]:
from collections import Counter
import pandas as pd
import json
import re

# Attempting to read the file using 'cp949' encoding commonly used for Korean text
file_path = '공동번역.txt'
json_output_path  = '공동번역_수정1.json'
csv_output_path = '공동번역_수정필요3.csv'


# Mapping for book abbreviations
book_mapping = {
    "01": "창", "02": "출", "03": "레", "04": "민", "05": "신",
    "06": "수", "07": "삿", "08": "룻", "09": "삼상", "10": "삼하",
    "11": "왕상", "12": "왕하", "13": "대상", "14": "대하", "15": "스",
    "16": "느", "17": "에", "18": "욥", "19": "시", "20": "잠",
    "21": "전", "22": "아", "23": "사", "24": "렘", "25": "애",
    "26": "겔", "27": "단", "28": "호", "29": "욜", "30": "암",
    "31": "옵", "32": "욘", "33": "미", "34": "나", "35": "합",
    "36": "습", "37": "학", "38": "슥", "39": "말",
    "40": "마", "41": "막", "42": "눅", "43": "요", "44": "행",
    "45": "롬", "46": "고전", "47": "고후", "48": "갈", "49": "엡",
    "50": "빌", "51": "골", "52": "살전", "53": "살후", "54": "딤전",
    "55": "딤후", "56": "딛", "57": "몬", "58": "히", "59": "약",
    "60": "벧전", "61": "벧후", "62": "요일", "63": "요이", "64": "요삼",
    "65": "유", "66": "계"
}

json_data = []
filtered_data = []

# Function to preprocess text
def preprocess_text(text):
    # If the text starts with '[' and contains ']', and doesn't contain 'ㄱ'
    if text.startswith('[') and ']' in text and 'ㄱ' not in text:
        # Remove text from '[' to the first ']'
        text = re.sub(r'^\[.*?\]\s*', '', text)
    # Remove leading spaces
    text = text.lstrip()
    # Replace terms
    text = text.replace("하느님", "하나님").replace("세례", "침례")
    return text

# Process the text file
with open(file_path, 'r', encoding='cp949') as file:
    lines = file.readlines()
    current_book = None

    for line in lines[2:]:  # Skip the first two lines
        parts = line.strip().split(" ", 2)
        if len(parts) < 3:
            continue

        book_number, chapter_verse, text = parts
        if current_book != book_number:
            current_book = book_number
            continue  # Skip the first line of each new book

        # Map book number to its abbreviation
        book_name = book_mapping.get(book_number, book_number)

        # Preprocess the text
        text = preprocess_text(text)

        # Extract chapter and verse
        try:
            chapter, verse = map(int, chapter_verse.split(":"))
        except ValueError:
            continue

        # Add to JSON data
        json_data.append({
            "book": book_name,
            "btext": text,
            "chapter": chapter,
            "id": len(json_data) + 1,
            "verse": verse
        })

        # Add to filtered data if it contains "[" or "ㄱ)"
        if "[" in text or "ㄱ)" in text:
            filtered_data.append({
                "book": book_name,
                "chapter": chapter,
                "verse": verse,
                "text": text
            })

# Save the JSON file
with open(json_output_path, 'w', encoding='utf-8') as json_file:
    json.dump(json_data, json_file, ensure_ascii=False, indent=4)

# Save the filtered data to CSV
filtered_df = pd.DataFrame(filtered_data)
filtered_df.to_csv(csv_output_path, index=False, encoding='utf-8-sig')

print("Processing complete!")
print(f"JSON file saved at: {json_output_path}")
print(f"CSV file saved at: {csv_output_path}")

Processing complete!
JSON file saved at: 공동번역_수정1.json
CSV file saved at: 공동번역_수정필요3.csv


In [4]:
edit_df = pd.read_csv('공동번역_수정완료.csv')
updated_json_path  = '공동번역_수정2.json'

# Replace the text in json_data based on book, chapter, and verse matches
for idx, row in edit_df.iterrows():
    book, chapter, verse, new_text = row['book'], row['chapter'], row['verse'], row['text']
    for entry in json_data:
        if entry['book'] == book and entry['chapter'] == chapter and entry['verse'] == verse:
            entry['btext'] = new_text  # Replace the text

# Save the updated JSON file
with open(updated_json_path, 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

print(f"Updated JSON saved at: {updated_json_path}")


Updated JSON saved at: 공동번역_수정2.json


In [7]:
filtered_csv_path = '공동번역_추가수정.csv'
selected_characters = r"[ㄱㄴㄷㄹㅁㅂㅅㅇㅈㅊ]"
filtered_verses = []

for entry in json_data:
    if re.search(selected_characters, entry['btext']):
        filtered_verses.append({
            "book": entry['book'],
            "chapter": entry['chapter'],
            "verse": entry['verse'],
            "text": entry['btext']
        })

# Convert to DataFrame and save to CSV
filtered_df = pd.DataFrame(filtered_verses)
filtered_df.to_csv(filtered_csv_path, index=False, encoding='utf-8-sig')

print(f"Updated JSON saved at: {updated_json_path}")
print(f"Filtered verses saved at: {filtered_csv_path}")


Updated JSON saved at: 공동번역_수정2.json
Filtered verses saved at: 공동번역_추가수정.csv


In [9]:
updates_csv_path = '공동번역_추가수정.csv'
updated_json_path = '공동번역_수정3.json'

updates_df = pd.read_csv(updates_csv_path)

# Replace the text in json_data based on book, chapter, and verse matches
for _, row in updates_df.iterrows():
    book, chapter, verse, new_text = row['book'], row['chapter'], row['verse'], row['text']
    for entry in json_data:
        if entry['book'] == book and entry['chapter'] == chapter and entry['verse'] == verse:
            entry['btext'] = new_text  # Replace the text

            # Preprocess the text again for json_data
            for entry in json_data:
                entry['btext'] = preprocess_text(entry['btext'])

# Save the updated JSON file
with open(updated_json_path, 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)

print(f"Updated JSON saved at: {updated_json_path}")


Updated JSON saved at: 공동번역_수정3.json


In [22]:
import requests
from bs4 import BeautifulSoup

def crawl_page(url):
    # Fetch the webpage
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Attempting to find a div containing the main Bible text. Adjust as needed:
    text_container = soup.find('div', id='tdBible1')
    if not text_container:
        # If this doesn't exist, try a different approach or print the entire HTML to inspect.
        print("Could not find the main text container. Printing partial HTML for inspection:")
        print(soup.prettify()[:1000])  # print first 1000 characters for a quick check
        return

    # Extract text, stripping excess whitespace
    bible_text = text_container.get_text(strip=True, separator=' ')

    return bible_text

if __name__ == "__main__":
    url = "https://www.bskorea.or.kr/bible/korbibReadpage.php?version=COG&book=gen&chap=1&sec=1&cVersion=&fontSize=15px&fontWeight=normal"
    text = crawl_page(url)
    if text:
        print("Extracted text:")
        print(text)


Extracted text:
공동번역 창세기 제 1 장 천지창조 1 1) 한 처음에 하느님께서 하늘과 땅을 지어 내셨다. "하느님께서 하늘과 땅을 지어 내시던 한 처음이었다" 라고 옮길 수도 있음. 2 땅은 아직 모양을 갖추지 않고 아무 것도 생기지 않았는데, 어둠이 깊은 물 위에 뒤덮여 있었고 그 물 위에 하느님의 2) 기운이 휘돌고 있었다. "바람" "영" "혼" "얼"이라고도 옮길 수 있음. 3 하느님께서 "빛이 생겨라!" 하시자 빛이 생겨났다. 4 그 빛이 하느님 보시기에 좋았다. 하느님께서는 빛과 어둠을 나누시고 5 빛을 낮이라, 어둠을 밤이라 부르셨다. 이렇게 첫날이 밤, 낮 하루가 지났다. 6 하느님께서 "물 한가운데 창공이 생겨 물과 물 사이가 갈라져라!" 하시자 그대로 되었다. 7 하느님께서는 이렇게 창공을 만들어 창공 아래 있는 물과 창공 위에 있는 물을 갈라 놓으셨다. 8 하느님께서 그 창공을 하늘이라 부르셨다. 이렇게 이튿날도 밤, 낮 하루가 지났다. 9 하느님께서 "하늘 아래 있는 물이 한 곳으로 모여, 마른 땅이 드러나거라!" 하시자 그대로 되었다. 10 하느님께서는 마른 땅을 뭍이라, 물이 모인 곳을 바다라 부르셨다. 하느님께서 보시니 참 좋았다. 11 하느님께서 "땅에서 푸른 움이 돋아나거라! 땅 위에 낟알을 내는 풀과 씨 있는 온갖 과일나무가 돋아나거라!" 하시자 그대로 되었다. 12 이리하여 땅에는 푸른 움이 돋아났다. 낟알을 내는 온갖 풀과 씨 있는 온갖 과일나무가 돋아났다. 하느님께서 보시니 참 좋았다. 13 이렇게 사흗날도 밤, 낮 하루가 지났다. 14 하느님께서 "하늘 창공에 빛나는 것들이 생겨 밤과 낮을 갈라 놓고 절기와 나날과 해를 나타내는 표가 되어라! 15 또 하늘 창공에서 땅을 환히 비추어라!" 하시자 그대로 되었다. 16 하느님께서는 이렇게 만드신 두 큰 빛 가운데서 더 큰 빛은 낮을 다스리게 하시고 작은 빛은 밤을 다스리게 하셨다. 또 별들도 만드셨다. 17 하느님께서는 이 빛나는 것들을 하늘 창공에 걸어 놓

In [18]:
import re

raw_text = text

def extract_verses(text):
    # Remove the initial header part if needed. We assume verses start at "1 1)" or "1 "
    # Let's try to isolate the part starting from verse 1:
    # We'll look for the first occurrence of "1 " which should mark verse 1.
    start_idx = text.find("1 ")
    if start_idx == -1:
        # If not found, try "1)"
        start_idx = text.find("1)")
    if start_idx != -1:
        text = text[start_idx:].strip()

    # The verses in Genesis 1 are numbered 1 to 31.
    # We can split by a regex pattern that identifies the start of each verse.
    # Pattern explanation:
    # (?=(\d{1,2}\s))  - A look-ahead for one or two digits followed by a space, marking verse start
    # We'll also consider that some verses might have a format like "1 1)" at the start.
    # Let's just find all occurrences of verses followed by a space or space+digit or space+parenthesis.
    # A robust pattern might be: `(?=\d+\s*\d?\))|(?=\d+\s)`

    # We'll first try a simpler approach: split on space before a verse number at word boundaries.
    # Since each verse likely starts with a number followed by a space, we can try:
    verses = re.split(r'(?=\b\d{1,2}\s)', text)

    # This might leave an empty string at the start if text started with verse 1.
    verses = [v.strip() for v in verses if v.strip()]

    # Now each element in verses should start with something like "1 1) 한 처음에..."
    # We need to extract the verse number and the text.
    result = []
    id_counter = 1
    for v in verses:
        # Verse format might be: "1 1) 한 처음에..." or "2 땅은...".
        # Let's extract the leading verse number.
        match = re.match(r'^(\d+)\s*(\d?\)?)\s*(.*)$', v)
        # Explanation:
        # ^(\d+)    : capture one or more digits (verse number)
        # \s*(\d?\)?) : optionally capture something like "1)" or just ")"
        # \s*(.*)$  : capture the rest of the line as the verse text
        
        if match:
            verse_num = int(match.group(1))
            verse_text = match.group(3).strip()

            # The verse_text may contain additional notes or parentheses. 
            # If you want to remove those notes like '2)', you can do further cleaning.
            # For now, let's just leave it as is or remove trailing notes if needed.

            # Create the dictionary entry
            verse_dict = {
                "book": "창",
                "btext": verse_text,
                "chapter": 1,
                "id": id_counter,
                "verse": verse_num
            }
            result.append(verse_dict)
            id_counter += 1

    return result


if __name__ == "__main__":
    verse_list = extract_verses(raw_text)
    for verse in verse_list:
        print(verse)


{'book': '창', 'btext': '장 천지창조', 'chapter': 1, 'id': 1, 'verse': 1}
{'book': '창', 'btext': '한 처음에 하느님께서 하늘과 땅을 지어 내셨다. "하느님께서 하늘과 땅을 지어 내시던 한 처음이었다" 라고 옮길 수도 있음.', 'chapter': 1, 'id': 2, 'verse': 1}
{'book': '창', 'btext': '땅은 아직 모양을 갖추지 않고 아무 것도 생기지 않았는데, 어둠이 깊은 물 위에 뒤덮여 있었고 그 물 위에 하느님의 2) 기운이 휘돌고 있었다. "바람" "영" "혼" "얼"이라고도 옮길 수 있음.', 'chapter': 1, 'id': 3, 'verse': 2}
{'book': '창', 'btext': '하느님께서 "빛이 생겨라!" 하시자 빛이 생겨났다.', 'chapter': 1, 'id': 4, 'verse': 3}
{'book': '창', 'btext': '그 빛이 하느님 보시기에 좋았다. 하느님께서는 빛과 어둠을 나누시고', 'chapter': 1, 'id': 5, 'verse': 4}
{'book': '창', 'btext': '빛을 낮이라, 어둠을 밤이라 부르셨다. 이렇게 첫날이 밤, 낮 하루가 지났다.', 'chapter': 1, 'id': 6, 'verse': 5}
{'book': '창', 'btext': '하느님께서 "물 한가운데 창공이 생겨 물과 물 사이가 갈라져라!" 하시자 그대로 되었다.', 'chapter': 1, 'id': 7, 'verse': 6}
{'book': '창', 'btext': '하느님께서는 이렇게 창공을 만들어 창공 아래 있는 물과 창공 위에 있는 물을 갈라 놓으셨다.', 'chapter': 1, 'id': 8, 'verse': 7}
{'book': '창', 'btext': '하느님께서 그 창공을 하늘이라 부르셨다. 이렇게 이튿날도 밤, 낮 하루가 지났다.', 'chapter': 1, 'id': 9, 'verse': 8}
{'bo